In [1]:
import os

In [2]:
%pwd

'd:\\Project\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Project\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
import os
from pathlib import Path
import tensorflow as tf
from dataclasses import dataclass
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            #UPDATED TO YOUR REPO
            mlflow_uri="https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

# ---------------------------------------------------------
# 3. EVALUATION CLASS
# ---------------------------------------------------------
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()

        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        
        dagshub.init(
            repo_owner='aryan18s', 
            repo_name='Kidney-Disease-Classification-Deep-Learning-Project', 
            mlflow=True
        )
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


d:\Anaconda\envs\kidney\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [6]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    print("✅ Evaluation and MLflow logging completed successfully!")
except Exception as e:
    raise e

[2026-02-23 22:30:35,508: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-23 22:30:35,516: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-23 22:30:35,518: INFO: common: created directory at: artifacts]
[2026-02-23 22:30:36,019: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 139 images belonging to 2 classes.
9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9568 - loss: 0.2136
[2026-02-23 22:30:50,752: INFO: common: json file saved at: scores.json]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

d:\Anaconda\envs\kidney\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d3f30528-36e3-4e38-9d10-c7f00fe41368&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4b0c4118038e287acdcfb811b81f23b30b77f1e86899cd53d75fc79ea7eea01d


[2026-02-23 22:31:56,058: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2026-02-23 22:31:57,721: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2026-02-23 22:31:58,907: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


...

[2026-02-23 22:32:01,218: INFO: helpers: Initialized MLflow to track repo "aryan18s/Kidney-Disease-Classification-Deep-Learning-Project"]


Repository aryan18s/Kidney-Disease-Classification-Deep-Learning-Project initialized!

[2026-02-23 22:32:01,221: INFO: helpers: Repository aryan18s/Kidney-Disease-Classification-Deep-Learning-Project initialized!]


2026/02/23 22:32:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/23 22:32:06 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Successfully registered model 'VGG16Model'.
2026/02/23 22:33:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run secretive-cod-914 at: https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/98d75f4fe47d4a7bb573f9b438604aa0
🧪 View experiment at: https://dagshub.com/aryan18s/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0
✅ Evaluation and MLflow logging completed successfully!
